# Ising-Lenz Ergodicity: Rate of convergence to approach to ergodicity field dependence

    
     (c) 2013, 2014, 2015, 2016, 2025 Süzen
     GPL v3 

This notebook generates Ising-Lenz 1D, rate of convergence to ergodicity data across temperatures. 
The dataset is called `ising1DrateErgodicityFields.rds`. 

$\Omega_{G}(t)$ : Modified TM G-metric, ergodic convergence.  
$\Gamma_{G}(t)$ : Rate of ergodic convergence.  
$\Kappa_{G}(t)$ : Inverse rate of ergodic convergence.   

Here we compute $\Gamma$ in a modified fashion, original portion is that ensemble average is 
an exact expression. 

* Workhorse of the data generation is method `performIS_repeat`. The RDS file is simply 
an object of list, each member is the output of this method, a named list. See return of this method.
Average magnetisations are reported there, 10 repeats
* Parameters are hardcoded in paralell method below.
     * Sizes at Metropolis (`transP=1`)  and Glauber (`transP=2`) Dynamics `N=512, 1024, 1536`.
     * Field ranges `seq(from = 0.5, by = 0.06, length.out = 16)`.
     * `J=1.0 ikBT=1.0`.
     * MC steps `5e6`.


In [25]:
rm(list=ls()) 
require("isingLenzMC"); # Developed and tested on v0.2.5 and R v4.5.1
library(parallel)       # Core package

source("../src/power_utilities.R");

In [26]:
parallel_gamma_field <- function(size, dynamics = 1, num_cores = 8) {
    # Parallel runs
    nfield <- 16
    max_mc_time <- 5e6
    fields <- seq(from = 0.5, by = 0.06, length.out = nfield)
    mc_gamma <- function(field) {
        performIS_repeat(ikBT =1.0, J = 1.0, H = field, size, max_mc_time, dynamics, nrepeat = 40)
    }

    current_field <- 1
    results <- vector("list", nfield)
    cycle <- as.integer(nfield / num_cores)
    field_count <- 0

    for (i in 1:cycle) {
        fromField <- field_count + 1
        toField <- field_count + num_cores
        par_result <- mclapply(fields[fromField:toField], mc_gamma, mc.cores = num_cores)
        for (j in 1:num_cores) {
            results[[current_field]] <- par_result[[j]]
            current_field <- current_field + 1
        }
        field_count <- field_count + num_cores
    }
    results
}


In [27]:
# ~200 min on 8 core MacBook-M2
result <- parallel_gamma_field(512, dynamics = 1)
result <- c(result, parallel_gamma_field(512, dynamics = 2))
result <- c(result, parallel_gamma_field(1024, dynamics = 1))
result <- c(result, parallel_gamma_field(1024, dynamics = 2))
result <- c(result, parallel_gamma_field(1536, dynamics = 1))
result <- c(result, parallel_gamma_field(1536, dynamics = 2))

In [28]:
saveRDS(result, "../datasets/ising1DrateErgodicityFields.rds")